<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Olhando-para-o-dataset-Portland" data-toc-modified-id="Olhando-para-o-dataset-Portland-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Olhando para o dataset Portland</a></span></li><li><span><a href="#Exemplo-Sintético" data-toc-modified-id="Exemplo-Sintético-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exemplo Sintético</a></span></li><li><span><a href="#Para-pensar" data-toc-modified-id="Para-pensar-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Para pensar</a></span></li><li><span><a href="#Referências" data-toc-modified-id="Referências-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Referências</a></span></li></ul></div>

# Olhando para o dataset Portland

Vamos dar uma olhada no dataset que enviamos para vocês. Primeiro, olharemos a coluna **Month** para ver se há valores repetidos, o que poderia ser um problema, já que iremos usá-la como index.

In [71]:
import pandas as pd

PATH_PORT = 'portland-oregon.csv'

df_portland = pd.read_csv(PATH_PORT)

print('Infos do dataset')
print(df_portland.info())

quant_dup = df_portland.loc[:, 'Month'].duplicated().sum()
print(f'\nQuant de duplicados em Month: {quant_dup}')

Infos do dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Month   114 non-null    object
 1   Count   114 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB
None

Quant de duplicados em Month: 0


Vamos criar uma outra célula, com o código completo, escolhendo a coluna **Month** como index já em tempo de leitura. Lembre-se, estamos usando o Jupyter-Notebook, apesar de ele ser otimo para explorações rápidas, não é a melhor ferramenta para desenvolvermos uma aplicação completa para Data Science, maaaaas continuemos.

In [72]:
import pandas as pd

PATH_PORT = 'portland-oregon.csv'

df_portland = pd.read_csv(PATH_PORT, index_col='Month', parse_dates=['Month'])
print(df_portland)

            Count
Month            
1960-01-01    648
1960-02-01    646
1960-03-01    639
1960-04-01    654
1960-05-01    630
...           ...
1969-02-01   1425
1969-03-01   1419
1969-04-01   1432
1969-05-01   1394
1969-06-01   1327

[114 rows x 1 columns]


Vamos pensar, os valores que estão na coluna **Count** são contínuos, mais ainda, ela é a única coluna que temos, ou seja, não temos uma coluna de **features** - que são as variáveis - e uma coluna de **Target** - aquilo que queremos prever.<br>

Na verdade, olhando mais de perto, vemos que a coluna **Count** é a coluna de **Target**, já que ela contêm os valores que queremos prever, a quantidade de pessoas que usaram o transporte público no mês seguinte. Como ela tem valores continuos, temos um problema de **REGRESSÃO**.<br>

O que temos até agora? 
- A coluna de **Targets**
- Um problema de Machine Learning Supervisionado e de **REGRESSÃO**

Então, onde estão as **features**?<br>

Antes, vamos citar aqui uma parte do artigo [What is Time Series Forecasting?](https://machinelearningmastery.com/time-series-forecasting/)

> Time series adds an explicit order dependence between observations: a time dimension. This additional dimension is both a constraint and a structure that provides a source of additional information.<br>
"A time series is a sequence of observations taken sequentially in time."<br>
— Page 1, [Time Series Analysis: Forecasting and Control.](https://www.amazon.com/Time-Analysis-Forecasting-Probability-Statistics/dp/1118675029/ref=as_li_ss_tl?dchild=1&keywords=Time+Series+Analysis:+Forecasting+and+Control&qid=1597365982&sr=8-1&linkCode=sl1&tag=inspiredalgor-20&linkId=8aacd7138dd0383953e41674e84246f4&language=en_US)

# Exemplo Sintético

Vou criar um exemplo sintético e vocês aplicam as ideias no dataset real. Suponha que queremos treinar um ESTIMATOR do Scikit-Learn para ele aprendar a contar de 1 em 1. Temos um dataset que vai de 1 até 10. Veja abaixo

In [70]:
import pandas as pd
import numpy as np

df_sum = pd.DataFrame(np.arange(1, 11), columns=['Count'])
print(df_sum)

   Count
0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10


Esse nosso problema é basicamente um problema de Time Series já que as observações têm uma "sequência" no tempo. Eu sei, tem coisas mais profundas, mas no final das contas - para efeitos práticos - podemos entender assim. Sabemos também que podemos abordá-lo como um problema de Machine Learning Supervisionado e de Regressão, mas que precisamos "cutucar" para que ele fique de uma maneira consumível para um ESTIMATOR, a saber:
- Design Matrix X 2D(n_samples, n_features)
- Target Values y 1D(n_samples, )

Citando o artigo [Time Series Forecasting as Supervised Learning](https://machinelearningmastery.com/time-series-forecasting-supervised-learning/)
> Given a sequence of numbers for a time series dataset, we can restructure the data to look like a supervised learning problem. We can do this by using previous time steps as input variables and use the next time step as the output variable.

Ou seja, a coluna **Count** é tanto o **Target Values y** quanto a **Design Matrix X**, que terá quantas features quisermos. Usando apenas uma feature: 
- temos um target value 2, cuja feature é 1
- temos um target value 3, cuja feature é 2
- temos um tatget value 4, cuja feature é 3
- etc etc etc

Caso quiséssemos duas features:
- temos um target value 3, cujas feature são 1 e 2
- temos um target value 4, cujas feature são 2 e 3
- temos um target value 5, cujas feature são 3 e 4
- etc etc etc

O nome dessa técnica é **Sliding Window**

Vamos colocar na prática no nosso dataset sintético. Vamos usar apenas uma feature e inserir uma coluna de Target y, onde o valor de y é o valor seguinte da coluna Count, esta - por sua vez - fica sendo a única feature.

In [73]:
import pandas as pd
import numpy as np

# cria dataset sintetico
df_sum = pd.DataFrame(np.arange(1, 11), columns=['Count'])

# inserimos uma coluna Target que eh o valor seguinte
# na sequencia
df_sum.insert(loc=1, column='Target', value=df_sum.Count.shift(-1))

print('Dataset refeito com a Design Matrix X e Target Values y')
print(df_sum)

Dataset refeito com a Design Matrix X e Target Values y
   Count  Target
0      1     2.0
1      2     3.0
2      3     4.0
3      4     5.0
4      5     6.0
5      6     7.0
6      7     8.0
7      8     9.0
8      9    10.0
9     10     NaN


Caso usássemos duas features, ficaria assim. Count é a primeira feature e Feature_2 a segunda, obviamente. Perceba que a ordem no tempo das observações foi preservada.

In [74]:
import pandas as pd
import numpy as np

# cria dataset sintetico
df_sum = pd.DataFrame(np.arange(1, 11), columns=['Count'])

df_sum.insert(loc=1, column='Feature_2', value=df_sum.Count.shift(-1))
df_sum.insert(loc=2, column='Target', value=df_sum.Count.shift(-2))

print('Dataset refeito com a Design Matrix X e Target Values y')
print(df_sum)

Dataset refeito com a Design Matrix X e Target Values y
   Count  Feature_2  Target
0      1        2.0     3.0
1      2        3.0     4.0
2      3        4.0     5.0
3      4        5.0     6.0
4      5        6.0     7.0
5      6        7.0     8.0
6      7        8.0     9.0
7      8        9.0    10.0
8      9       10.0     NaN
9     10        NaN     NaN


Veja que sempre uma observação anterior, ou um conjunto delas, é a Design Matrix X e a observação seguinte é o Target Values y

# Para pensar
- como faríamos o train_test_split nesse caso?  lembre-se que a ordem no tempo das observações deve ser preservada
- o que faremos com os valores NaN que surgiram no dataset?

# Referências
- [What Is a Time Series Forecasting?](https://machinelearningmastery.com/time-series-forecasting/)
- [Time Series Forecasting as Supervised Learning](https://machinelearningmastery.com/time-series-forecasting-supervised-learning/)
- [Forecasting: Principles and Practice](https://otexts.com/fpp2/)